In [11]:
# -*- coding: utf-8 -*
import twitter
import csv
import json
import re
import requests
import lxml.html as html
from urllib2 import urlopen
import pandas as pd
from datetime import datetime, timedelta
from dateutil import tz
import time
import sklearn.cross_validation as cv
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
%matplotlib inline

In [12]:
OUT_NEWS_FILE = "news.csv"
OUT_TWITTER_FILE = "twitter.csv"

In [13]:
class TJLoader:

    def __init__(self):
        self._news_pages = ["https://tjournal.ru/paper/page/{}"]#, "https://tjournal.ru/club/news/recent/page/{}"]
        self._month_map = {u"января":"01", u"февраля":"02", u"марта":"03", u"апреля":"04", u"мая":"05", u"июня":"06", u"июля":"07", u"августа":"08", u"сентября":"09", u"октября":"10", u"ноября":"11", u"декабря":"12"}

    def get_news_uri(self, min_index=10, count=30):
        """

        :param min_index: int, индекс страницы, с которой нужно начать поиск
        :param count: int, количество страниц, которые нужно скачать
        :return: list. список ссылок на новости
        """
        links = []
        for news_page in self._news_pages:
            
            for i in range(count):
                page = html.parse(urlopen(news_page.format(i+min_index)))
                divs = page.getroot().find_class('b-articles__b__title')

                for div in divs:
                    links.append(div.getchildren()[1].get("href"))
                    
        return links

    
    def _parse_date(self, date):
        date = date.replace(",", "")
        date = date.split(" ")
        
        converted_date = date[2]
        converted_date +="-"+self._month_map[date[1]]
        converted_date +="-"+date[0]
        
        converted_date +=" "+date[3]
        
        return converted_date
        

    def get_link_info(self, link):
        """

        :param link: str, url страницы с tjournal, для которой нужно собрать информацию
        :return: dict с данными со страницы
        """
        page = html.parse(urlopen(link))
        root = page.getroot()

        # заголовок
        title = root.find_class("b-article__title")
        title = title[0].find("h1").text

        # парсим количество просмотров
        view = root.get_element_by_id("hitsCount").text
        view = view.replace(" ", "")
        view = int(view)

        # Количество комментариев
        comments = root.find_class("b-article__infoline__comments")
        comment = int(comments[0].find("b").text.replace(" ", ""))

        # Теги
        tags = root.find_class("b-article__tags__tag")
        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        # Дата
        date = root.find_class("b-article__infoline__date")
        date = self._parse_date(date[0].text)
        
        # Первоисточник
        source = root.find_class("b-article__link")
        if len(source) != 0:
            source = source[0].getchildren()[0].getchildren()[1].getchildren()[0].text
        else:
            source = None
        
        # Type
        if "tjournal.ru/c/" in link:
            news_type = "TJ_C"
        else:
            news_type = "TJ_P"
            
        return {
            "url": link, 
            "title": title, 
            "views": view, 
            "comments": comment, 
            "tags": tag_list, 
            "date": date, 
            "type": news_type, 
            "source": source}


    def get_tj_news_info(self, min_index=1, count=30, first_date="2017-01-01", last_date="2010-01-01"):
        """
        :param min_index: int, индекс минимальной страницы, откуда начинаем поиск
        :param count: int, количество страниц, по которым ищем
        :first_date: время первой новости, которую мы скачаем
        :last_date: время последней новости, которую мы скачаем
        :return: dict с данными со страницы
        """
        links = self.get_news_uri(min_index=min_index, count=count)
        link_info_list = []
        i = 0
        is_break = False
        for link in links:
            link_info = self.get_link_info(link)
            # Если заданное время не подходит
            if link_info["date"] > first_date or link_info["date"] < last_date:
                continue
    
            link_info_list.append(link_info)
            
            i+=1
            if i%10 == 0:
                print "Скачали ", i, " страниц"

        return link_info_list

In [19]:
class VCLoader:

    def __init__(self):
        self._news_pages = "https://api.vc.ru/1/paper"
        self._month_map = {u"января":"01", u"февраля":"02", u"марта":"03", u"апреля":"04", u"мая":"05", u"июня":"06", u"июля":"07", u"августа":"08", u"сентября":"09", u"октября":"10", u"ноября":"11", u"декабря":"12"}

    def get_news_uri(self, min_index=10, count=30):
        """

        :param min_index: int, индекс страницы, с которой нужно начать поиск
        :param count: int, количество страниц, которые нужно скачать
        :return: list. список ссылок на новости
        """
        links = []
        text = requests.get(self._news_pages).text
        json_req = json.loads(text)
    
        for news in json_req:
            links.append(news["url"])

        return links

    
    def _parse_date(self, date):
        date = date.replace(",", "")
        date = date.split(" ")
        
        converted_date = date[2]
        converted_date +="-"+self._month_map[date[1]]
        converted_date +="-"+date[0]
        
        converted_date +=" "+date[3]
        
        return converted_date
        

    def get_link_info(self, link):
        """

        :param link: str, url страницы с vc, для которой нужно собрать информацию
        :return: dict с данными со страницы
        """
        page = html.parse(urlopen(link))
        root = page.getroot()

        # заголовок
        title = root.find_class("b-article__head")
        title = title[0].find("h1").text

        # парсим количество просмотров
        view = root.get_element_by_id("hitsCount").text
        view = view.replace(" ", "")
        view = int(view)

        # Количество комментариев
        comments = root.find_class("ccount")[0].text
        comment = int(comments.replace(" ", ""))

        # Теги
        tags = root.find_class("b-tags__tag")
        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        # Дата
        date = root.find_class("b-article__infopanel__date")
        date = self._parse_date(date[0].text)
        

        return {"title": title, "views": view, "comments": comment, "tags": tag_list, "date": date, "url": link, "type": "VC"}


    def get_tj_news_info(self, min_index=1, count=30, first_date="2010-01-01", last_date="2017-01-01"):
        """
        :param min_index: int, индекс минимальной страницы, откуда начинаем поиск
        :param count: int, количество страниц, по которым ищем
        :first_date: str, дата и время первой (самой новой) новости
        :last_date: str, дата и время последней(самой старой) новости
        
        :return: dict с данными со страницы
        """
        links = self.get_news_uri(min_index=min_index, count=count)
        link_info_list = []
        i = 0
        
        for link in links:
            
            link_info = self.get_link_info(link)
            # Если заданное время не подходит
            if link_info["date"] > first_date or link_info["date"] < last_date:
                continue
            
            link_info_list.append(link_info)
            
            i+=1
            if i%10 == 0:
                print "Скачали ", i, " страниц"

        return link_info_list

In [15]:
class RSSLoader:
    
    def __init__(self):
        #self._pages = ["https://roem.ru/rss/roem-all-news.xml", "http://lifenews.ru/xml/feed.xml", "http://www.forbes.ru/newrss.xml", "http://rg.ru/xml/index.xml", "http://www.vesti.ru/vesti.rss", "http://lenta.ru/rss", "http://ria.ru/export/rss2/index.xml"]
        self._pages = ["https://roem.ru/rss/roem-all-news.xml", "http://lifenews.ru/xml/feed.xml", "http://www.forbes.ru/newrss.xml", "http://www.vesti.ru/vesti.rss", "http://lenta.ru/rss", "http://ria.ru/export/rss2/index.xml"]
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}
        
        
        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"
        
        
    def _parse_date(self, date):

        """

        :param date: str, дата в формате - "Fri, 04 Dec 2015 14:45:39 +0000"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        timezone = date.split("+")[1]
        date = date.replace(",", "")
        date_array = date.split(' ')
        
        time = date_array[4]
        day = int(date_array[1])
        month = self._month_dict[date_array[2]]
        year = date_array[3]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        
        if (timezone != "0300"):
            utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
            date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        # Обрезаем зону
        date = str(date).split("+")[0]
        
        # Обрезаем секунды
        splited = date.split(":")
        date = splited[0]+":"+splited[1]
        return date
    
    
    def _get_type(self, url):
        url = url.replace("www", "")
        url = url.split("://")[1]
        url = url.split("/")[0]
        return url
        
    
    def _get_hours_until_now(self, date):
        news_date = datetime.strptime(date, '%Y-%m-%d %H:%M')
        now = datetime.today()
        return float((now-news_date).total_seconds()/3600)
        
    def get_news_array(self, hour=8):
        """
        hour: int, время последней новости
        """
        news_array = []
        for url in self._pages:
            news_type = self._get_type(url)
            print url
            
            tree = ET.ElementTree(file=urlopen(url))
            root = tree.getroot()
            for i in root.iter('item'):
                link = i.find('link').text
                title = i.find('title').text
                date_ = i.find('pubDate').text
                date = self._parse_date(date_)
                
                news_info =  {
                    "title": title,
                    "date": date, 
                    "url": link, 
                    "type": news_type
                }
                
                if self._get_hours_until_now(date) >= hour:
                    news_array.append(news_info)
                
            
        print "Собрано ", len(news_array), " новостей с RSS"
        return news_array

In [16]:
class TwitterLoader:


    def __init__(self):
        CONSUMER_KEY = 'BOuuaMDhNhm6yx0rzqK8bMsbI'
        CONSUMER_SECRET = '3DybJwlkXd2vU6R385yLA8yJblYJltLtwojySD9AVs04ShauZ0'

        ACCESS_TOKEN_KEY = '3712177576-of3jzZ8gNmlPDfPjPyR0Ljw1Ao2IXdTqX9dZGDZ'
        ACCESS_TOKEN_SECRET = 'Ky7iKwByHNXX3UMfuMhv6UgVx2IhjLo3KmwpsBQz35wtG'

        self.api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}

        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"



    def _parse_date(self, date):

        """

        :param date: str, дата в формате твиттера - "Sat Nov 21 17:00:29 +0000 2015"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        date_array = date.split(' ')
        month = self._month_dict[date_array[1]]
        day = int(date_array[2])
        time = date_array[3]
        year = date_array[5]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
        moscow_date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        return str(moscow_date).split("+")[0]



    def loadTweetWithLink(self, link, date):
        """

        :param link: str, ключевое слово для поиска, вданном случае - ссылка на письмо
        :param date: str, дата, до которой искать твиты
        :return: список словариков с информацией о УНИКАЛЬНЫХ твитах по запросу link
        """
        set_id = set()
        tweet_list = []
        result = self.api.GetSearch(term=link, until=date, count=100000)

        for res in result:

            tw_id = res.GetId()
            # Если мы уже обрабатывали этот твит - идем дальше
            if tw_id in set_id:
                continue

            retweeted_status = res.GetRetweeted_status()
            # Если это не ретвит
            if retweeted_status is None:
                is_retweet = 0
                retweeted_count = res.GetRetweetCount()
                favorite_count = res.GetFavoriteCount()
            else:
                is_retweet = 1
                retweeted_count = 0
                favorite_count = 0
            set_id.add(tw_id)

            created_at = res.GetCreatedAt()
            created_at = self._parse_date(created_at)

            # Данные о пользователе
            user = res.GetUser()
            followers_count = user.followers_count
            listed_count = user.listed_count
            friends_count = user.friends_count
            favourites_count = user.favourites_count
            statuses_count = user.statuses_count


            tw_dict= {
                    "url": link,
                    "tw_id":tw_id,
                    "retweeted_count": retweeted_count,
                    "favorite_count":favorite_count,
                    "is_retweet": is_retweet,
                    "created_at":created_at,
                    "user_followers_count":followers_count,
                    "user_listed_count": listed_count,
                    "user_friends_count":friends_count,
                    "user_favourites_count":favourites_count,
                    "user_statuses_count":statuses_count
                    }

            tweet_list.append(tw_dict)

        return tweet_list

    def _get_next_date(self, date, days):
        """

        :param date: str, дата
        :param days: int, количество дней
        :return: возвращает дату через days-дней после date
        """
        date = datetime.strptime(date, '%Y-%m-%d %H:%M')
        date += timedelta(days=days)
        return str(date).split(' ')[0]


    def load_tweets_by_term(self, news_list, days_after_news=2):
        """

        :param news_list: list(str), список словариков с информацией о новости
        :param days_after_news:  int, количество дней после публикации новости, до которой искать
        """
        result_list = []
        i = 0
        for news in news_list:
            tweets = []
            try:
                until_date = self._get_next_date(news["date"], days_after_news)
                tweets = self.loadTweetWithLink(news["url"], until_date)
            except twitter.error.TwitterError as ex:
                print str(ex)
                if str(ex) == self._RATE_LIMIT:
                    sleep_time = self.api.GetSleepTime("search/tweets") #??? Почему-то не работает
                    print "Спим {} сек.".format(sleep_time)
                    time.sleep(sleep_time+2)
                    tweets = self.loadTweetWithLink(news["url"], until_date)

            result_list+= tweets
            
            i+=1
            if i%10 == 0:
                print "Собрано информация о", i, " новостях"
            
        return result_list

### Подготавливаем  данные для анализа

In [77]:
news_info = pd.read_csv(OUT_NEWS_FILE, sep=",")
tweeter_data = pd.read_csv(OUT_TWITTER_FILE, sep=",")

In [78]:
news_info.head(10)

,comments,date,source,tags,title,type,url,views
0,51,2015-12-05 18:45,NaN,"[подписка, мобильные приложения, урбанизация, ...",Мнение: Приложения и интернет-сервисы убивают ...,VC,https://vc.ru/p/app-around-corner,6412
1,3,2015-12-05 16:10,NaN,"[bluetooth, iBeacon, bluetooth-маяки]","iBeacon-маяки для бизнеса: кейсы от EasyJet, B...",VC,https://vc.ru/p/ibeacon-marketing,1644
2,26,2015-12-05 14:17,NaN,"[Россия, стартапы, китай, рейтинг, индия, мала...","Россия, Польша, Тунис и другие — топ-10 стран ...",VC,https://vc.ru/p/best-cities-for-startups,3950
3,0,2015-12-05 13:07,NaN,"[Игры, книги, безбумажная библиотека, nintendo...",«Консольные войны»: Sega против Nintendo,VC,https://vc.ru/p/sega-nintendo,1817
4,7,2015-12-04 21:23,NaN,[uber],Украинское министерство инфраструктуры анонсир...,VC,https://vc.ru/n/uber-ukraine,3367
5,7,2015-12-06 00:21,NaN,"[Breaking Mad, депутаты, драматичные новости]",Драматичные новости недели от Breaking Mad: шт...,TJ_P,https://tjournal.ru/p/breaking-mad-cientotres,2233
6,21,2015-12-05 19:05,NaN,"[музыка, безопасность, Википедия, знаменитости]","Австралиец попал за кулисы к любимой группе, и...",TJ_P,https://tjournal.ru/p/peking-duk-fan,17766
7,23,2015-12-05 14:01,NaN,"[фото, Instagram, Новый год, Рождество]",Праздник к нам приходит: ёлки в разных странах...,TJ_P,https://tjournal.ru/p/christmas-trees-around-t...,5326
8,88,2015-12-05 12:40,NaN,"[СМИ, журналистика, медиа, Алексей Венедиктов,...",Леся Рябцева объявила об уходе с «Эха Москвы» ...,TJ_P,https://tjournal.ru/p/lesya-echo-off,45124
9,169,2015-12-04 22:20,NaN,"[научные исследования, британские учёные, глуп...",Глупость любителей «глубокомысленных» цитат по...,TJ_P,https://tjournal.ru/p/stupid-quotes,27746


In [79]:
# !!!!   !!!!!!!!!! !!!!!!!!!!1 !!!!!!!!!!!!
# Сначала почищу данные. Там, где не материалы редакции - мусор, на который твитов то почти и нет!
news_info = news_info[ (news_info['type'] != "TJ_C")]
len(news_info["url"].unique())

721

In [80]:
# Мерджим дата фреймы
df = news_info.merge(tweeter_data, on='url', left_index=True, right_index=False)

In [81]:
df.head(1)

,comments,date,source,tags,title,type,url,views,created_at,favorite_count,is_retweet,retweeted_count,tw_id,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,51,2015-12-05 18:45,NaN,"[подписка, мобильные приложения, урбанизация, ...",Мнение: Приложения и интернет-сервисы убивают ...,VC,https://vc.ru/p/app-around-corner,6412,2015-12-06 12:01:57,0,1,0,673427142642921472,8450,469,153,23,75097


In [82]:
def diff_date_minutes(news_date, tweet_date):
    news_date = datetime.strptime(news_date, '%Y-%m-%d %H:%M')
    tweet_date = datetime.strptime(tweet_date, '%Y-%m-%d %H:%M:%S')
    return int((tweet_date-news_date).total_seconds()/60)

In [83]:
# количество минут, с момента публикации записи, данного твита
df["time_since_news"] = df.apply(lambda s: diff_date_minutes(s["date"], s["created_at"]), axis=1)

In [84]:
def get_week_day(date):
    date = datetime.strptime(date, '%Y-%m-%d %H:%M')
    return date.weekday()

In [85]:
# индекс дня недели
df["week_day_news"] = df.date.apply(lambda s: get_week_day(s))

In [86]:
def get_minutes_since_midnight(date):
    midnight = date.split(" ")[0] + " 00:00"
    date = datetime.strptime(date, '%Y-%m-%d %H:%M')
    midnight = datetime.strptime(midnight, '%Y-%m-%d %H:%M')
    return int((date-midnight).total_seconds()/60)

In [87]:
# Сколько минут времени прошло с полуночи
df["minutes_since_midnight"] = df.date.apply(lambda s: get_minutes_since_midnight(s))

### Агрегируем данные из твиттера

In [88]:
# Время для вычисления независимых переменных
FIRST_TIME = 10
# Время для вычисления целевой функции
LAST_TIME = 180

In [89]:
len(df["url"].unique())

691

In [90]:
ft_df = df[df["time_since_news"] <= FIRST_TIME]

In [91]:
grouped = ft_df.groupby("url")

In [92]:
# Считаем общее количество твиттов
count_of_tweets = pd.DataFrame(grouped["url"].count())
count_of_tweets.columns = ["first_time_tweet"]
count_of_tweets.reset_index(inplace=True)  
df = pd.merge(df, count_of_tweets, on='url', left_index=True, right_index=False, how="outer")

In [93]:
# Считаем общую аудиторию
follower_sum = pd.DataFrame(grouped["user_followers_count"].sum())
follower_sum.columns = ["follower_sum"]
follower_sum.reset_index(inplace=True)  
df = pd.merge(df, follower_sum, on='url', left_index=True, right_index=False, how="outer")

In [94]:
# Считаем число ретвитов
retweeted_count_sum = pd.DataFrame(grouped["retweeted_count"].sum())
retweeted_count_sum.columns = ["retweeted_count_sum"]
retweeted_count_sum.reset_index(inplace=True)  
df = pd.merge(df, retweeted_count_sum, on='url', left_index=True, right_index=False, how="outer")

In [95]:
# Считаем число звездочек
favorite_count_sum = pd.DataFrame(grouped["favorite_count"].sum())
favorite_count_sum.columns = ["favorite_count_sum"]
favorite_count_sum.reset_index(inplace=True)  
df = pd.merge(df, favorite_count_sum, on='url', left_index=True, right_index=False, how="outer")

In [96]:
# Считаем общее число списков, в которых состоят сделавшие посты
user_listed_count = pd.DataFrame(grouped["user_listed_count"].sum())
user_listed_count.columns = ["user_listed_count_sum"]
user_listed_count.reset_index(inplace=True)  
df = pd.merge(df, user_listed_count, on='url', left_index=True, right_index=False, how="outer")

In [97]:
st_df = df[df["time_since_news"] <= LAST_TIME]

In [98]:
# Выделяем значение целевой функции
grouped = st_df.groupby("url")
count_of_tweets = pd.DataFrame(grouped["url"].count())
count_of_tweets.columns = ["last_time_tweet"]
count_of_tweets.reset_index(inplace=True)  
df = pd.merge(df, count_of_tweets, on='url', left_index=True, right_index=False, how="outer")

In [99]:
PREPARED_CSV = "prepared_to_analys.csv"

In [100]:
# Сохраняемся
df.to_csv(PREPARED_CSV, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [101]:
len(df["url"].unique())

691

# После первого запуска, через несколько дней, запускать только часть, которая находится под этим заголовком!

In [17]:
HOURS = 8 # Количество часов от текущего момента. Текущее время - Hours часов - время последней собранной новости

In [18]:
def get_hours_ago(hours=3):
    d = datetime.now() - timedelta(hours=hours)
    d = str(d).split(":")
    d = d[0]+":"+d[1]
    return d


def load_new_news(news_df, hours=3):
    # Вычисляем самую молодую новость
    tj_last_date = (news_df[news_df["type"]=="TJ_P"]["date"]).max()
    #tj_c_last_date = (news_df[news_df["type"]=="TJ_C"]["date"]).max()
    
    #tj_last_date = max(tj_p_last_date, tj_c_last_date)
    vc_last_date = (news_df[news_df["type"]=="VC"]["date"]).max()
    first_date = get_hours_ago(hours)
    
    
    print "Качаем новости с ", tj_last_date," и ", vc_last_date, " по ", first_date
    
    tj_loader = TJLoader()
    tj_pages = tj_loader.get_tj_news_info(min_index=0, count=4, first_date=first_date, last_date=tj_last_date)    
    
    vc_loader = VCLoader()
    vc_pages = vc_loader.get_tj_news_info(first_date=first_date, last_date=vc_last_date)
    
    pages = vc_pages + tj_pages
    pages
    
    return pages

# Загружаем данные
news_prev_df = pd.read_csv(OUT_NEWS_FILE, sep=",")
twitter_prev_df = pd.read_csv(OUT_TWITTER_FILE, sep=",")

# Качаем новости с VC и TJ
pages = load_new_news(news_prev_df, HOURS)
# Качаем новости RSS
rss_loader = RSSLoader()
rss_pages = rss_loader.get_news_array()
# Объединяем
pages += rss_pages

tw = TwitterLoader()
tweets = tw.load_tweets_by_term(pages)

# Объединяем новости с предыдущими
news = pd.DataFrame(pages)
news = (news.append(news_prev_df)).reset_index(drop=True)

# Объединяем твиты с предыдущими
tweets_df = pd.DataFrame(tweets)
tweets_df = (tweets_df.append(twitter_prev_df)).reset_index(drop=True)

# Сохраняем
tweets_df.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
news.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

Качаем новости с  2015-12-04 22:20  и  2015-12-04 21:23  по  2015-12-06 06:13
Скачали  10  страниц
https://roem.ru/rss/roem-all-news.xml
http://lifenews.ru/xml/feed.xml
http://www.forbes.ru/newrss.xml
http://www.vesti.ru/vesti.rss
http://lenta.ru/rss
http://ria.ru/export/rss2/index.xml
Собрано  394  новостей с RSS
Собрано информация о 10  новостях
Собрано информация о 20  новостях
Собрано информация о 30  новостях
Собрано информация о 40  новостях
Собрано информация о 50  новостях
Собрано информация о 60  новостях
Собрано информация о 70  новостях
Собрано информация о 80  новостях
Собрано информация о 90  новостях
Собрано информация о 100  новостях
Собрано информация о 110  новостях
Собрано информация о 120  новостях
Собрано информация о 130  новостях
Собрано информация о 140  новостях
Собрано информация о 150  новостях
Собрано информация о 160  новостях
Собрано информация о 170  новостях
Собрано информация о 180  новостях
[{u'message': u'Rate limit exceeded', u'code': 88}]
Спим 724 се

# УДАЛЯЕМ ПОВТОРЫ =(((

In [38]:
# Сначала для твиттера
tw_data = pd.read_csv(OUT_TWITTER_FILE, sep=",")
last_size = len(tw_data)
dupl = tw_data["tw_id"].duplicated()
dupl = np.invert((dupl.as_matrix()))
tw_data = tw_data[dupl]
print "Удалено ", last_size-len(tw_data)

Удалено  384


In [39]:
# Сохраняем!
tw_data.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [40]:
# Теперь для новостей
news_df = pd.read_csv(OUT_NEWS_FILE, sep=",")
last_size = len(news_df)
dupl = news_df["url"].duplicated()
dupl = np.invert((dupl.as_matrix()))
news_df = news_df[dupl]
print "Удалено ", last_size-len(news_df)

Удалено  8


In [41]:
# Сохраняем
news_df.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

# ======================================

In [76]:
df["type"].unique()

array(['VC', 'TJ_P'], dtype=object)

# Здесь уже нет ничего хорошего. Уходи

In [ ]:
news = news.append(news_df)

In [ ]:
# читаем данные
news_df = pd.read_csv(OUT_NEWS_FILE, sep=",")
twitter_df = pd.read_csv(OUT_TWITTER_FILE, sep=",")

In [ ]:
# находим самую "молодую новость"
news_df = news_df.sort_values(by=["date"], ascending=False)
tj_last_date = (news_df[news_df["type"]=="TJ_P"]["date"]).max()
vc_last_date = (news_df[news_df["type"]=="VC"]["date"]).max()

In [ ]:
vc_last_date

In [ ]:
def get_tj_news_info(self, min_index=1, count=30, first_date="2015-11-25 12:58", last_date="2015-11-29 12:58"):

In [ ]:
# скачивае информацию о новостях
loader = TJLoader()
pages = loader.get_tj_news_info(min_index=0, count=30, first_date="2015-11-25 12:58", last_date="2015-11-29 12:58")

In [ ]:
len(pages)

In [ ]:
# Сохраняем в файлик
news_df = news_df.append(pd.DataFrame(pages))
news_df.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# Скачиваем данные из твиттера
tw = TwitterLoader()
tweeter_data = tw.load_tweets_by_term(pages)

In [ ]:
len(tweeter_data)

In [ ]:
# Сохраняем в файлик
twitter_df = twitter_df.append(pd.DataFrame(tweeter_data))
twitter_df.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
date = datetime.now()

In [ ]:
d = datetime.now() - timedelta(hours=2)

In [ ]:
d = str(d).split('.')[0]

In [ ]:
import json

In [ ]:
text = requests.get("https://api.vc.ru/1/paper").text

In [ ]:
json_req = json.loads(text)

In [ ]:
page = html.parse(urlopen("https://vc.ru/p/interview-it"))
root = page.getroot()
print root

In [ ]:
page = html.parse(urlopen("https://vc.ru/n/microsoft-store-russia"))
root = page.getroot()

# заголовок
title = root.find_class("b-article__head")
title = title[0].find("h1").text
print title

# парсим количество просмотров
view = root.get_element_by_id("hitsCount").text
view = view.replace(" ", "")
view = int(view)
print view

# Количество комментариев
comments = root.find_class("ccount")[0].text
comment = int(comments.replace(" ", ""))
print comment

tags = root.find_class("b-tags__tag")
tag_list = []
for tag in tags:
    tag_list.append(tag.text)
    
print tag_list

In [ ]:
print title

In [ ]:
vc = VCLoader()

In [ ]:
vc.get_link_info("https://vc.ru/n/microsoft-store-russia")

In [ ]:
vc_dict = vc.get_tj_news_info()

Вопросы:

1) Что делать с соурсе? Эти новости никто не репостит с TJournal, а от других источников их можно найти

2) Урлы для других запросов на VC. Не хотелось бы их брудфорсить.